In [0]:
# Importações necessárias

from pyspark.sql.functions import from_unixtime, col, round, to_date

In [0]:
# Leitura dos dados bronze

df = spark.table("steam.bronze.user_games")

In [0]:
# Converte para timestamp
df = df.withColumn(
    'rtime_last_played',
    from_unixtime(col('rtime_last_played')).cast('timestamp')
)

# Conversão minutos → horas com 2 casas decimais
df = df.withColumn(
    'playtime_forever',
    round(col('playtime_forever') / 60, 2).cast('double')
)

# Filtrar apenas valores maiores que 0.00
df = df.filter(col('playtime_forever') > 0.00)

# Convertendo coluna string para int
df = df.withColumn("appid", col("appid").cast("int"))

# Selecionando colunas ao dataframe final
df = df.select('appid','playtime_forever','rtime_last_played')

In [ ]:
%sql
-- Usando SQL mágico do Databricks para criar schema se não existir
CREATE SCHEMA IF NOT EXISTS steam.silver.user_games

In [ ]:
%sql
-- Criação da tabela Delta se não existir
CREATE TABLE IF NOT EXISTS steam.silver.user_games (
  appid INT,
  playtime_forever DOUBLE,
  rtime_last_played TIMESTAMP
)
USING DELTA

In [0]:
# Escrita na camada silver
df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("steam.silver.user_games")